In [178]:
import numpy as np
from scipy.integrate import solve_ivp

import plotly.graph_objects as go

In [179]:
# RK45; Adapted from https://stackoverflow.com/questions/54494770/how-to-set-fixed-step-size-with-scipy-integrate
def DoPri45Step(f,t,x,h):

    k1 = f(t,x) 
    k2 = f(t + 1./5*h, x + h*(1./5*k1) ) 
    k3 = f(t + 3./10*h, x + h*(3./40*k1 + 9./40*k2) ) 
    k4 = f(t + 4./5*h, x + h*(44./45*k1 - 56./15*k2 + 32./9*k3) ) 
    k5 = f(t + 8./9*h, x + h*(19372./6561*k1 - 25360./2187*k2 + 64448./6561*k3 - 212./729*k4) ) 
    k6 = f(t + h, x + h*(9017./3168*k1 - 355./33*k2 + 46732./5247*k3 + 49./176*k4 - 5103./18656*k5) )

    v5 = 35./384*k1 + 500./1113*k3 + 125./192*k4 - 2187./6784*k5 + 11./84*k6
    k7 = f(t + h, x + h*v5)
    v4 = 5179./57600*k1 + 7571./16695*k3 + 393./640*k4 - 92097./339200*k5 + 187./2100*k6 + 1./40*k7

    return v4,v5

def DoPri45integrate(f, t, x0):
    N=len(t)
    x = [x0]
    for k in range(N-1):
        v4, v5 = DoPri45Step(f,t[k],x[k],t[k+1]-t[k])
        x.append(
            x[k] + (t[k+1]-t[k])*v5
        )
    return np.array(x)

In [180]:
# Explicit Euler

def ExplicitEulerStep(f, t, x, h):
    return f(t, x)

def ExplicitEulerIntegrate(f, t, x0):
    N = len(t)
    x = [x0]
    for k in range(N-1):
        v1 = ExplicitEulerStep(f, t[k], x[k], t[k+1]-t[k])
        x.append(
            x[k] + (t[k+1]-t[k])*v1
        )
    return np.array(x)

In [182]:
## PROBLEM DEFINITION
# x = y[0], y = y[1]
# Initial Conditions
y0 = [1,0]
# PARAMS
epsilons = [0.16, 0.01, 0.0025, 0.0009, 0.000625]

In [183]:
# SOLVER DEFINITION
h = 1e-3
ts = np.arange(0,10,h)
solvers = [
    {'func':ExplicitEulerIntegrate, 'name': 'Explicit Euler'}, 
    {'func':DoPri45integrate, 'name': 'Dormand-Prince-45 (Explicit RKM of Order 4(5))'}
]

solutions = []

In [184]:
for solver in solvers:
    for epsilon in epsilons:
        # ODE
        f = lambda t,y: np.array([
            (-y[1] + y[0]*(1 - y[0])**2)/epsilon,
            y[0]
        ])
        solutions.append({
            'ys':solver['func'](f, ts, y0).T, 
            'ts':ts,
            'name':solver['name'],
            'epsilon':epsilon,
        })

C:\Users\NENNIA\AppData\Local\Continuum\miniconda3\envs\test\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\NENNIA\AppData\Local\Continuum\miniconda3\envs\test\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\NENNIA\AppData\Local\Continuum\miniconda3\envs\test\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning:

invalid value encountered in subtract

C:\Users\NENNIA\AppData\Local\Continuum\miniconda3\envs\test\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in subtract

C:\Users\NENNIA\AppData\Local\Continuum\miniconda3\envs\test\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in add

C:\Users\NENNIA\AppData\Local\Continuum\miniconda3\envs\test\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning:

invalid value encountered in add

C:\Users\NENNIA\AppData\Local\Continuum\min

In [185]:
# RKM 45, Wanted to use explicit Jacobian, but its too ill-conditioned
for epsilon in epsilons:
    print("Epsilon: ", epsilon)
    # ODE
    f = lambda t,y: np.array([
        (-y[1] + y[0]*(1 - y[0])**2)/epsilon,
        y[0]
     ])
    # Solution
    sol = solve_ivp(
        fun=f, 
        t_span=[0,10],
        t_eval=ts,
        y0=y0,
        method='Radau',
        dense_output=False,
    )
    print(sol.message)
    
    solutions.append({
        'ts': sol.t,
        'ys': sol.y,
        'name':'Radau(Variable Step Size, Implicit RKM of Order 5)',
        'epsilon':epsilon,
    })

Epsilon:  0.16
Required step size is less than spacing between numbers.
Epsilon:  0.01
Required step size is less than spacing between numbers.
Epsilon:  0.0025
Required step size is less than spacing between numbers.
Epsilon:  0.0009
Required step size is less than spacing between numbers.
Epsilon:  0.000625
Required step size is less than spacing between numbers.


In [186]:
figs = []
for s in solutions:
    #print(s)
    fig = go.Figure(
        data=go.Scatter(x=s['ys'][0], y=s['ys'][1])
    )
    fig.update_layout(
        title="Solver: {0}, ε={1}".format(s['name'], s['epsilon'])
    )
    
    key = (s['name'], s['epsilon'])
    fig.write_image("images/van-der-pol-%s-%s.pdf"%key)
    figs.append(fig)

In [187]:
for fig in figs:
    key
    fig.write_image("images/van-der-pol-%s-%s.pdf"%key)